In [2]:
import pandas as pd
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import json
from bs4 import BeautifulSoup
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException


In [3]:
from bs4 import BeautifulSoup
import json
import urllib
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from selenium.common.exceptions import NoSuchElementException

In [4]:
driver = webdriver.Firefox(executable_path=r'C:\geckodriver.exe')

def openbrowser(locid, key):
    driver.wait = WebDriverWait(driver, 5)
    driver.maximize_window()
    words = key.split()
    txt =''    
    for w in words:
        txt +=(w+'+')
    #print (txt)
    driver.get("https://www.glassdoor.co.in/Job/jobs.htm?suggestCount=0&suggestChosen=true&clickSource=searchBtn&typedKeyword={}
               &sc.keyword={}&locT=C&locId={}&jobType=fulltime&fromAge=1&radius=6&cityId=-1&minRating=0.0&industryId=-1
               &sgocId=-1&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0".format(txt[:-1],txt[:-1], locid))
    return driver

def geturl(driver):
    url = set()
    while True:
        print(len(url))
        if len(url)>=20:
            break
        soup1 = BeautifulSoup(driver.page_source, "lxml")
        
        main = soup1.find_all("li",{"class":"jl"})
        
        for m in main:
            url.add('https://www.glassdoor.co.in{}'.format(m.find('a')['href']))       
        try:
            next_element = soup1.find("li", {"class": "next"})
            try:
                next_exist = next_element.find('a')
            except AttributeError:
                driver.quit()
                break
            except NoSuchElementException:
                driver.quit()
                break
            if next_exist:
    
                driver.find_element_by_class_name("next").click()
                time.sleep(2)
            else:
                driver.quit()
                break
        except ElementClickInterceptedException:
            pass
        
    return list(url)

x =openbrowser(locid =4477468, key='"software testing"')
with open('url_software_testing_loc_bangalore.json','w') as f:
    json.dump(geturl(driver),f, indent = 4)
    print("file created")

<selenium.webdriver.firefox.webdriver.WebDriver (session="fe9a5b0c-e9f3-4173-8ff8-99aad7d012cb")>
0
30
file created


In [ ]:


with open('url_software_testing_loc_bangalore.json','r') as f:
    url = json.load(f)
data ={}    
i = 1
jd_df = pd.DataFrame()
driver = webdriver.Firefox(executable_path=r'C:\geckodriver.exe')
for u in tqdm(url):
    driver.wait = WebDriverWait(driver, 2)
    driver.maximize_window()
    driver.get(u)
    soup = BeautifulSoup(driver.page_source, "lxml")
    try:
       
        header = soup.find("div",{"class":"header cell info"})
        position = driver.find_element_by_tag_name('h2').text
        company = driver.find_element_by_xpath("//span[@class='strong ib']").text
        location = driver.find_element_by_xpath("//span[@class='subtle ib']").text
        jd_temp = driver.find_element_by_id("JobDescriptionContainer")
        jd = jd_temp.text
        info = soup.find_all("infoEntity")
    except IndexError:
        print('IndexError: list index out of range')
    except NoSuchElementException:
        pass
    data[i] = {
        'url' :u,
        'Position':position,
        'Company': company,
        'Location' :location,
        'Job_Description' :jd
    }
    i+=1     
driver.quit()
jd_df = pd.DataFrame(data)
jd = jd_df.transpose()

jd = jd[['url','Position','Company','Location','Job_Description']]
jd.to_csv('jd_software_testing_banglore.csv')
print('file created')

In [2]:
import pandas as pd

In [3]:
df1=pd.read_csv("jd_software_testing.csv")
df1

,url,Position,Company,Location,Job_Description
0,https://www.glassdoor.co.in/partner/jobListing...,Data Scientist,Citibank,â€“ Bengaluru,About us:\n\nGlobal Decision Management (GDM) ...
1,https://www.glassdoor.co.in/partner/jobListing...,KGS:: MC:: Data Modeler,KPMG,â€“ Bengaluru,Roles and Responsibilities:\n\nModel data sour...
2,https://www.glassdoor.co.in/partner/jobListing...,Data Engineer,Knoema,â€“ Bengaluru,Requirements\n2+ years of experience in DBMS d...
3,https://www.glassdoor.co.in/partner/jobListing...,Data Scientist,GO-JEK,â€“ Bengaluru,Responsibilities: Work as part of a product te...
4,https://www.glassdoor.co.in/partner/jobListing...,"Data Scientist, Sr. II",Lam Research,â€“ Bengaluru,Eligibility Criteria:\n\nBachelorâ€™s in Engin...
...,...,...,...,...,...
294,https://www.glassdoor.co.in/partner/jobListing...,Sr Staff Data Engineer,General Electric,â€“ Bengaluru,Role Summary:\n\nThe Sr Staff Data Engineer wi...
295,https://www.glassdoor.co.in/partner/jobListing...,Senior Data Scientist,Nanobi Data And Analytics,â€“ Bengaluru,Designation : Senior Data Scientist\n\nExperie...
296,https://www.glassdoor.co.in/partner/jobListing...,Data Scientist,KaHa Pte,â€“ Bengaluru,"Responsibilities:\n\nBuild, validate, test, an..."
297,https://www.glassdoor.co.in/partner/jobListing...,Principal Data Scientist,Unbxd Inc,â€“ Bengaluru,Unbxd is an AI-driven eCommerce Search Platfor...
